In [ ]:
%matplotlib inline
import cv2
from theano import *
theano.config.openmp = True

from keras import *
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import seaborn as sns
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import *
from keras.applications.vgg16 import VGG16

In [ ]:
fobia_folder = "./trypophobic/"
normal_folder = "./data_train/reddit_sub_pics/"
resolution_x = 225
resolution_y = 225
def load_folder(folder):
    
    images = []
    
    if(not os.path.isdir(folder[:-1]+"-parsed")):
        os.mkdir(folder[:-1]+"-parsed")
    
    for file in tqdm(os.listdir(folder)):
        image = cv2.imread(os.path.join(folder, file))
        if(image is not None):
            image = cv2.resize(image, (resolution_x, resolution_y), interpolation=cv2.INTER_LINEAR)
            cv2.imwrite(os.path.join(folder[:-1]+"-parsed/",file), image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image.reshape(3,resolution_x, resolution_y))
    images = np.array(images)
    return images

In [ ]:
fobia_images = load_folder(fobia_folder).astype(np.float32) /255.
normal_images = load_folder(normal_folder).astype(np.float32) /255.

In [4]:
data_y = [] # 1,0 -> fobia | 0,1 -> normal
data_x = []
for img in fobia_images:
    data_x.append(img)
    data_y.append([1,0])
for img in normal_images:
    data_x.append(img)
    data_y.append([0,1])

data_x = np.array(data_x)
data_y = np.array(data_y)

X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=4)

datagen = ImageDataGenerator(
   rotation_range=20,
   horizontal_flip=True,
   dim_ordering="th")

In [5]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [6]:
model = models.Sequential()
#model.add(layers.Convolution2D(nb_filter=1, nb_col=1, nb_row=1,
#                               input_shape=(3,resolution_x,resolution_y),
#                               dim_ordering="th"))
#model.add(layers.Activation(activation="relu"))
#---------------------------------------------
model.add(layers.Convolution2D(nb_filter=8, nb_col=10, nb_row=10, 
                               dim_ordering="th", 
                               input_shape=(3,resolution_x,resolution_y)))
model.add(layers.Activation(activation="relu"))
model.add(layers.MaxPooling2D(dim_ordering="th", pool_size=(5,5)))
model.add(layers.Dropout(0.1))
#---------------------------------------------
model.add(layers.Convolution2D(nb_filter=8, nb_col=3, nb_row=3,
                              dim_ordering="th"))
model.add(layers.Activation(activation="relu"))
model.add(layers.MaxPooling2D(dim_ordering="th", pool_size=(5,5)))
model.add(layers.Dropout(0.1))
#---------------------------------------------
model.add(layers.Convolution2D(nb_filter=8, nb_col=3, nb_row=3,
                              dim_ordering="th"))
model.add(layers.Activation(activation="relu"))
#---------------------------------------------
model.add(layers.Convolution2D(nb_filter=8, nb_col=3, nb_row=3,
                              dim_ordering="th"))
model.add(layers.Activation(activation="relu"))
#---------------------------------------------
model.add(layers.GlobalMaxPooling2D(dim_ordering="th"))
#model.add(layers.MaxPooling2D(dim_ordering="th", pool_size=(3,3)))
#model.add(layers.Convolution2D(nb_filter=8, nb_col=3, nb_row=3))
#model.add(layers.Activation(activation="relu"))
#model.add(layers.MaxPooling2D(dim_ordering="th", pool_size=(3,3)))
#model.add(layers.Flatten())
#model.add(layers.Dense(16))
#model.add(layers.Activation(activation="relu"))
model.add(layers.Dense(2))
model.add(layers.Activation(activation="softmax"))

In [ ]:
model = VGG16(weights='imagenet', include_top=False)

In [ ]:
optimizer = "adam"#optimizers.SGD(lr=0.1, momentum=0.01, nesterov=True)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=8),
                    samples_per_epoch=len(X_train), nb_epoch=5, validation_data=(X_test, Y_test))

Epoch 1/5
1248/1248 [==============================] - 189s - loss: 0.6869 - acc: 0.5425 - val_loss: 0.6785 - val_acc: 0.5417

In [ ]:
model.layers[4]

In [ ]:
weights = model.layers[0].W.get_value(borrow=True).reshape(8,10,10,3)
print (weights.shape)
fig, axes = plt.subplots(2,8, sharex=True, sharey=True)
plt.rcParams['figure.figsize'] = (10,5)
for i,w in enumerate(weights):
    axes[int(i/8)][int(i%8)].imshow(w)
    if(i == 31):
        break
plt.show()

In [ ]:
np.random.seed(4*4*4)
sample_image = X_test[np.random.randint(0, len(X_test))]
plt.imshow(sample_image.reshape(resolution_x, resolution_y, 3))
plt.show()

In [ ]:
convout1_f = theano.function(model.inputs, [model.layers[0].output])
sample_outs = convout1_f([sample_image.astype(np.float32)])[0][0]

fig, axes = plt.subplots(4,8, sharex=True, sharey=True)
plt.rcParams['figure.figsize'] = (10, 5)
for i,w in enumerate(sample_outs):
    axes[int(i/8)][int(i%8)].imshow(w, cmap="gray")
    #sns.heatmap(w)
    #plt.show()
plt.show()